In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"bhaktiselokar","key":"96cd2544107f2c54f9d7d71309a3794d"}'}

In [2]:
import os
import zipfile
import tensorflow as tf

In [3]:
os.makedirs("/root/.kaggle", exist_ok = True)
!mv kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d bittlingmayer/amazonreviews

Dataset URL: https://www.kaggle.com/datasets/bittlingmayer/amazonreviews
License(s): unknown
 96% 471M/493M [00:01<00:00, 382MB/s]
100% 493M/493M [00:01<00:00, 466MB/s]


In [5]:
!unzip -q amazonreviews.zip

In [6]:
!ls

amazonreviews.zip  sample_data	test.ft.txt.bz2  train.ft.txt.bz2


In [47]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, f1_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, SimpleRNN, GRU, Bidirectional, LSTM, Dense, Dropout, LayerNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
import bz2
import csv
import re

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
with bz2.BZ2File("/content/train.ft.txt.bz2") as f:
    training_data = [f.readline().decode('utf-8').strip() for _ in range(3000)]

# Load a subset of the test data
with bz2.BZ2File("/content/test.ft.txt.bz2") as f:
    test_data = [f.readline().decode('utf-8').strip() for _ in range(1000)]

# Print the subset sizes
print("Training data subset size:", len(training_data))
print("Test data subset size:", len(test_data))

Training data subset size: 3000
Test data subset size: 1000


In [9]:
training_data[0]

'__label__2 Stuning even for the non-gamer: This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^'

In [30]:
def clean_text(text):
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english')) - set(['no', 'not', 'nor'])
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    words = text.split()
    words = [lemmatizer.lemmatize(w) for w in words if w not in stop_words]
    return " ".join(words)

In [31]:
training_labels = [0 if '__label__1' in line else 1 for line in training_data]
training_texts = [clean_text(re.sub(r'__label__\d ', '', line)) for line in training_data]

In [32]:
test_labels = [0 if '__label__1' in line else 1 for line in test_data]
test_texts = [clean_text(re.sub(r'__label__\d ', '', line)) for line in test_data]

In [33]:
max_words = 10000
max_seq_len = 100

In [34]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(training_texts)


In [35]:
X_train = pad_sequences(tokenizer.texts_to_sequences(training_texts), maxlen=max_seq_len)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_texts), maxlen=max_seq_len)
y_train, y_test = np.array(training_labels), np.array(test_labels)

In [37]:
# Download GloVe 100D (if not done already)
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip


--2025-07-27 11:50:15--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2025-07-27 11:50:15--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-07-27 11:50:16--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [38]:
embedding_index = {}
with open("glove.6B.100d.txt", encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefs

embedding_dim = 100
embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in tokenizer.word_index.items():
    if i < max_words:
        vec = embedding_index.get(word)
        if vec is not None:
            embedding_matrix[i] = vec

In [41]:
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(enumerate(class_weights))

In [43]:
model = Sequential([
    Embedding(input_dim=max_words, output_dim=embedding_dim, weights=[embedding_matrix],
              input_length=max_seq_len, trainable=False),
    Bidirectional(LSTM(64, return_sequences=True, recurrent_dropout=0.2)),
    LayerNormalization(),
    Dropout(0.3),
    Bidirectional(LSTM(64, recurrent_dropout=0.2)),
    LayerNormalization(),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=3e-4), metrics=['accuracy'])
model.build(input_shape=(None, max_seq_len))
model.summary()


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ (None, 100, 100)       │     1,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_4 (Bidirectional) │ (None, 100, 128)       │        84,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_8           │ (None, 100, 128)       │           256 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 100, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_5 (Bidirectional) │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_9           │ (None, 128)            │           256 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,192,129 (4.55 MB)

 Trainable params: 192,129 (750.50 KB)

 Non-trainable params: 1,000,000 (3.81 MB)

In [44]:
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(X_train, y_train, batch_size=64, epochs=30,
                    validation_data=(X_test, y_test),
                    callbacks=[early_stop],
                    class_weight=class_weights,
                    verbose=1)


Epoch 1/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 67s 1s/step - accuracy: 0.5330 - loss: 0.8031 - val_accuracy: 0.6390 - val_loss: 0.6341
Epoch 2/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 53s 1s/step - accuracy: 0.6194 - loss: 0.6564 - val_accuracy: 0.7040 - val_loss: 0.5860
Epoch 3/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 53s 1s/step - accuracy: 0.6691 - loss: 0.6153 - val_accuracy: 0.7360 - val_loss: 0.5378
Epoch 4/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 52s 1s/step - accuracy: 0.7364 - loss: 0.5403 - val_accuracy: 0.7690 - val_loss: 0.5072
Epoch 5/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 83s 1s/step - accuracy: 0.7590 - loss: 0.5055 - val_accuracy: 0.7590 - val_loss: 0.4991
Epoch 6/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 81s 1s/step - accuracy: 0.7832 - loss: 0.4797 - val_accuracy: 0.7760 - val_loss: 0.4855
Epoch 7/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 84s 1s/step - accuracy: 0.8117 - loss: 0.4402 - val_accuracy: 0.7850 - val_loss: 0.4583
Epoch 8/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 82s 1s/step - accuracy: 0.8139 - loss: 0.4358 - val_accuracy: 0.7800 - val_loss:

In [45]:
y_probs = model.predict(X_test)
y_pred = (y_probs > 0.5).astype(int)

32/32 ━━━━━━━━━━━━━━━━━━━━ 10s 268ms/step


In [48]:
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("AUC Score:", roc_auc_score(y_test, y_probs))
print("F1 Score:", f1_score(y_test, y_pred))


Confusion Matrix:
 [[382 116]
 [ 84 418]]
Classification Report:
               precision    recall  f1-score   support

           0       0.82      0.77      0.79       498
           1       0.78      0.83      0.81       502

    accuracy                           0.80      1000
   macro avg       0.80      0.80      0.80      1000
weighted avg       0.80      0.80      0.80      1000

AUC Score: 0.8843061488983823
F1 Score: 0.806949806949807


In [50]:
def predict_sentiment(text, model, tokenizer, max_seq_len=100):
    # Use the same cleaning as during training
    text = clean_text(text)

    # Convert to sequence
    sequence = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(sequence, maxlen=max_seq_len)

    # Predict
    prob = model.predict(padded)[0][0]

    # Decide label
    sentiment = "Positive 😊" if prob > 0.5 else "Negative 😡"
    return sentiment, round(float(prob), 4)


In [51]:
sample_sentences = [
    "I absolutely love this! It works perfectly.",
    "This is the worst thing I’ve ever bought.",
    "It's fine, not amazing, not bad either."
]

for sentence in sample_sentences:
    sentiment, score = predict_sentiment(sentence, model, tokenizer)
    print(f"Text: {sentence}\nSentiment: {sentiment} (Confidence: {score})\n")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 324ms/step
Text: I absolutely love this! It works perfectly.
Sentiment: Positive 😊 (Confidence: 0.967)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step
Text: This is the worst thing I’ve ever bought.
Sentiment: Negative 😡 (Confidence: 0.1706)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step
Text: It's fine, not amazing, not bad either.
Sentiment: Negative 😡 (Confidence: 0.0446)



In [53]:
import pickle

model.save("lstm_model.h5")

# Save the tokenizer
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)